In [15]:
import os
import tensorflow as tf

In [16]:
# os.chdir("/home/sakshmeno/Documents/GitHub/C-RNN-approach/Labels")
os.chdir("/Users/saksh.menon/Documents/GitHub/C-RNN-approach/manual_dataset")
with open("guillermo_branch_simple_insecure.c") as dataset_obj:
    codeLines = dataset_obj.read()

codeLines

'#include <stdio.h>\n\nint main(void)\n{\n    int result = 1;\n    if (result == 1)\n    {\n        printf("Executing critical code...\\n");\n    }\n    else\n    {\n        printf("Exiting out...\\n");\n        return 1;\n    }\n\n    return 0;\n}'

In [17]:
def code_preprocessing(file):
    with open(file) as dataset_obj:
        codeLines = dataset_obj.read()
    comment_lines = []
    raw_codeLines = codeLines.replace("\t","").split("\n")
    multi_line_flag = 0
    for line_number in range(len(raw_codeLines)):
        if multi_line_flag:
            if raw_codeLines[line_number].__contains__("*/"):
                multi_line_flag = 0
            comment_lines.append(line_number)
        elif raw_codeLines[line_number].__contains__("/*"):
            if raw_codeLines[line_number].startswith("/*") and not(raw_codeLines[line_number].__contains__("*/")):
                comment_lines.append(line_number)
                multi_line_flag = 1
            elif raw_codeLines[line_number].__contains__("/*") and not(raw_codeLines[line_number].startswith("/*")):
                if raw_codeLines[line_number].__contains__("*/"):
                    psuedo_multi_line_start = raw_codeLines[line_number].find("/*")
                    psuedo_multi_line_end = raw_codeLines[line_number].find("*/")
                    temporary_line = raw_codeLines[line_number][:psuedo_multi_line_start] + raw_codeLines[line_number][psuedo_multi_line_end+2:]
                    raw_codeLines[line_number] = temporary_line

        elif raw_codeLines[line_number].startswith("//"):
            comment_lines.append(line_number)
        elif raw_codeLines[line_number].__contains__("//"):
            comment_start = raw_codeLines[line_number].find('//')
            raw_codeLines[line_number] = raw_codeLines[line_number][:comment_start]
            
    comment_lines.reverse()
    for i in comment_lines:
        raw_codeLines.pop(i)

    def insert_space(string, index):
        string_copy = ""
        for i in range(len(string)):
            if i==(index):
                string_copy += " "
                string_copy += string[i]
                string_copy += " "
                continue
            string_copy += string[i]
        return string_copy

    def find_char_indices(input_string, char):
        indices = []
        replacement_token = 0
        for index, character in enumerate(input_string):
            if character == char:
                indices.append(index + 2*replacement_token)
                replacement_token+=1
        return indices

    def space_out(string, char):
        indices = find_char_indices(string, char)

        for i in indices:
            string = insert_space(string, i)
        return string

    # for line_number in range(len(raw_codeLines)):
    #     placeHolder = raw_codeLines[line_number]
        # placeHolder = space_out(placeHolder, ";")
        # placeHolder = space_out(placeHolder, "(")
        # placeHolder = space_out(placeHolder, ")")
        # placeHolder = space_out(placeHolder, ",")
        # if placeHolder.endswith(";"):
        #     raw_codeLines[line_number] = "<start> " + placeHolder.replace(";","<end>")
        # elif not(placeHolder.endswith(";")):
        #     raw_codeLines[line_number] = "<start> " + placeHolder + " <end>"

    return raw_codeLines


In [18]:
dataset = code_preprocessing('guillermo_branch_simple_insecure.c')
f = open('FILTERED_DATASET.txt','w')
for i in range(len(dataset)):
    f.write(dataset[i])
    f.write('\n')
f.close()

In [22]:
text_ds = tf.data.TextLineDataset("/Users/saksh.menon/Documents/GitHub/C-RNN-approach/manual_dataset/FILTERED_DATASET.txt").filter(lambda x: tf.cast(tf.strings.length(x), bool))

In [20]:
def custom_standardization(input_data):
  lowercase = tf.strings.lower(input_data)
  return tf.strings.regex_replace(lowercase,
                                  '[%s]' % re.escape(string.punctuation), '')


# Define the vocabulary size and the number of words in a sequence.
vocab_size = 4096
sequence_length = 10

# Use the `TextVectorization` layer to normalize, split, and map strings to
# integers. Set the `output_sequence_length` length to pad all samples to the
# same length.
vectorize_layer = tf.keras.layers.TextVectorization(
    standardize=custom_standardization,
    max_tokens=vocab_size,
    output_mode='int',
    output_sequence_length=sequence_length)

In [23]:
import string
import re
vectorize_layer.adapt(text_ds.batch(1024))

In [24]:
# Save the created vocabulary for reference.
inverse_vocab = vectorize_layer.get_vocabulary()
print(inverse_vocab[:20])

['', '[UNK]', '1', 'return', 'result', 'int', 'stdioh', 'printfexiting', 'printfexecuting', 'outn', 'mainvoid', 'include', 'if', 'else', 'critical', 'coden', '0']


In [25]:
# Vectorize the data in text_ds.
SEED = 42
AUTOTUNE = tf.data.AUTOTUNE
text_vector_ds = text_ds.batch(1024).prefetch(AUTOTUNE).map(vectorize_layer).unbatch()

In [26]:
sequences = list(text_vector_ds.as_numpy_iterator())
print(len(sequences))

15


In [28]:
for seq in sequences:
  print(f"{seq} \t=>\t {[inverse_vocab[i] for i in seq]}")

[11  6  0  0  0  0  0  0  0  0] 	=>	 ['include', 'stdioh', '', '', '', '', '', '', '', '']
[ 5 10  0  0  0  0  0  0  0  0] 	=>	 ['int', 'mainvoid', '', '', '', '', '', '', '', '']
[0 0 0 0 0 0 0 0 0 0] 	=>	 ['', '', '', '', '', '', '', '', '', '']
[5 4 2 0 0 0 0 0 0 0] 	=>	 ['int', 'result', '1', '', '', '', '', '', '', '']
[12  4  2  0  0  0  0  0  0  0] 	=>	 ['if', 'result', '1', '', '', '', '', '', '', '']
[0 0 0 0 0 0 0 0 0 0] 	=>	 ['', '', '', '', '', '', '', '', '', '']
[ 8 14 15  0  0  0  0  0  0  0] 	=>	 ['printfexecuting', 'critical', 'coden', '', '', '', '', '', '', '']
[0 0 0 0 0 0 0 0 0 0] 	=>	 ['', '', '', '', '', '', '', '', '', '']
[13  0  0  0  0  0  0  0  0  0] 	=>	 ['else', '', '', '', '', '', '', '', '', '']
[0 0 0 0 0 0 0 0 0 0] 	=>	 ['', '', '', '', '', '', '', '', '', '']
[7 9 0 0 0 0 0 0 0 0] 	=>	 ['printfexiting', 'outn', '', '', '', '', '', '', '', '']
[3 2 0 0 0 0 0 0 0 0] 	=>	 ['return', '1', '', '', '', '', '', '', '', '']
[0 0 0 0 0 0 0 0 0 0] 	=>	 ['', '',